# ctrip 데이터 수집하고 번역하기

In [ ]:
import requests
import pandas as pd
import json

# 요청 URL 및 헤더 설정
url = "https://m.ctrip.com/restapi/soa2/18109/json/getAttractionList"
headers = {
    'Content-Type': 'application/json'
}

attractions = []

# 여러 페이지 데이터 수집
for index in range(1, 11):
    print(index, end=',')
    payload = {
        "head": {
            "cid": "09031079213586682514",
            "ctok": "",
            "cver": "1.0",
            "lang": "01",
            "sid": "8888",
            "syscode": "999",
            "auth": "",
            "xsid": "",
            "extension": []
        },
        "scene": "online",
        "districtId": 1446512,
        "index": index,
        "sortType": 1,
        "count": 10,
        "filter": {
            "filterItems": []
        },
        "returnModuleType": "product"
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()
    attraction_list = data.get("attractionList", [])

    for item in attraction_list:
        card = item.get("card", {})
        attractions.append({
            "poiId": card.get("poiId"),
            "poiName": card.get("poiName"),
            "commentCount": card.get("commentCount"),
            "commentScore": card.get("commentScore"),
            "districtName": card.get("districtName"),
            "zoneName": card.get("zoneName"),
            "distanceStr": card.get("distanceStr"),
            "coverImageUrl": card.get("coverImageUrl"),
            "openStatus": card.get("openStatus"),
            "latitude": card.get("coordinate", {}).get("latitude"),
            "longitude": card.get("coordinate", {}).get("longitude"),
            "detailUrl": card.get("detailUrl")
        })

# 데이터프레임으로 변환 및 출력
df = pd.DataFrame(attractions)
df

In [ ]:
# prompt: df 내용 ctrip_3000 이름으로 엑셀, csv 저장

# Save the DataFrame to an Excel file
df.to_excel('ctrip_list.xlsx', index=False)

# Save the DataFrame to a CSV file
df.to_csv('ctrip_list.csv', index=False)

In [ ]:
# prompt: ctrip_3000,csv 파일을 읽어와서 df 에 저장

df = pd.read_csv('ctrip_list.csv')
df

In [ ]:
# df["poiName"].to_dict()

In [ ]:
# !pip install openai

In [ ]:
# from google.colab import userdata
# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
import sqlite3

# SQLite 데이터베이스 연결 및 테이블 저장
db_name = 'poi_database.db'
table_name = 'poi_data'

conn = sqlite3.connect(db_name)
try:
    # 데이터프레임을 SQLite 테이블에 저장
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    print(f"'{table_name}' 테이블이 '{db_name}' 데이터베이스에 성공적으로 저장되었습니다.")
except Exception as e:
    print(f"에러 발생: {e}")
finally:
    conn.close()

## OpenAI API를 활용한 번역

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm
tqdm.pandas()

# OpenAI 클라이언트 초기화
# client = OpenAI(api_key=OPENAI_API_KEY)
client = OpenAI()
client

In [ ]:
# 데이터베이스 설정
db_name = 'poi_database.db'
table_name = 'poi_data'
columns_to_translate = ['poiName', 'districtName', 'zoneName', 'distanceStr']

# 데이터베이스에 _ko 컬럼 추가
def add_translation_columns():
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    
    # 테이블 정보 조회
    cur.execute(f"PRAGMA table_info({table_name})")
    existing_columns = [row[1] for row in cur.fetchall()]

    # 필요한 _ko 컬럼 추가
    for col in columns_to_translate:
        translated_col = f"{col}_ko"
        if translated_col not in existing_columns:
            alter_query = f"ALTER TABLE {table_name} ADD COLUMN {translated_col} TEXT"
            cur.execute(alter_query)
            print(f"{translated_col} 컬럼 추가 완료.")

    conn.commit()
    conn.close()

# 번역 함수 (1행씩 처리)
def translate_and_update(row):
    try:
        # 번역할 텍스트를 json 형태로 구성 (NULL이 아닌 경우만)
        to_translate = {col: row[col] for col in columns_to_translate if pd.notna(row[col])}

        if not to_translate:
            return

        # API 요청
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": f"Translate the following Chinese place names to Korean. Return the result as JSON with each key suffixed by '_ko': {json.dumps(to_translate, ensure_ascii=False)}"
                }
            ],
            max_tokens=300,
            temperature=0.7
        )

        # 마크다운 제거 및 JSON 파싱
        response_text = completion.choices[0].message.content.strip()

        if response_text.startswith("```json"):
            response_text = response_text.replace("```json", "").replace("```", "").strip()

        # JSON 파싱
        response_json = json.loads(response_text)

        # 번역된 데이터베이스 업데이트
        update_query = f"""
        UPDATE {table_name}
        SET {', '.join([f"{key} = ?" for key in response_json.keys()])}
        WHERE poiId = ?
        """
        values = list(response_json.values()) + [row['poiId']]

        conn = sqlite3.connect(db_name)
        cur = conn.cursor()
        cur.execute(update_query, values)
        conn.commit()
        conn.close()

        print(f"poiId {row['poiId']} 업데이트 완료.")

    except Exception as e:
        print(f"Error at poiId {row['poiId']}: {e}")

In [ ]:
# 데이터베이스에 필요한 컬럼 추가
add_translation_columns()

In [ ]:
conn = sqlite3.connect(db_name)
query = f"""
SELECT * FROM {table_name}
WHERE zoneName_ko IS NOT NULL OR distanceStr_ko IS NOT NULL
"""
df = pd.read_sql(query, conn)
# conn.close()

In [ ]:
df

In [ ]:
# 데이터베이스 연결 및 번역이 필요한 데이터 불러오기
conn = sqlite3.connect(db_name)
query = f"""
SELECT * FROM {table_name}
WHERE zoneName_ko IS NULL OR distanceStr_ko IS NULL
LIMIT 10
"""
df = pd.read_sql(query, conn)
conn.close()

# 번역 및 업데이트 수행
df.apply(lambda row: translate_and_update(row), axis=1)

print("모든 행 업데이트 완료.")

In [ ]:
conn = sqlite3.connect(db_name)
query = f"""
SELECT * FROM {table_name}
WHERE zoneName_ko IS NOT NULL OR distanceStr_ko IS NOT NULL
"""
df = pd.read_sql(query, conn)
conn.close()

df

## 번역 결과 다시 저장

In [ ]:
# prompt: 위 df 엑셀, csv 파일로 저장할 것 파일 이름은 ctrip_jeju 로 할 것

df.to_csv('ctrip_jeju.csv')
# files.download('ctrip_jeju.csv')

df.to_excel('ctrip_jeju.xlsx')
# files.download('ctrip_jeju.xlsx')

In [ ]:
df['poiName_ko']